## Coursera Course Scraper

Sends requests to Coursera and parses out course information using open public API endpoints

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import requests
import logging
import time
import os

import pandas as pd

from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

In [ ]:
course_endpoint_url = "https://www.coursera.org/api/courses.v1?q=slug&slug={}"


In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

output_log_file_name = f"{'coursera-courses'}-{time.strftime('%Y%m%d-%H%M%S')}.log"
file_path = os.path.join(os.getcwd(), "logs", output_log_file_name)
fh = logging.FileHandler(file_path)
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

In [ ]:
def get_all_courses():
    """Get all course links from coursera

    Returns
    -------
    courses : list
        List of all course links.
    """
    sitemap_url = "https://www.coursera.org/sitemap~www~courses.xml"
    response = requests.get(sitemap_url)
    if response.status_code != 200:
        logging.error("Failed to get sitWemap")
        return []
    soup = BeautifulSoup(response.text, "lxml")
    courses = [loc.text for loc in soup.find_all("loc")]
    return courses

In [ ]:
def get_slug(url):
    """Get slug from course url
    
    Parameters
    ----------
    url : str
        Course url.
        
    Returns
    -------
    slug : str
        Course slug.    
    """
    return url.split("/")[-1]

In [ ]:
def get_course_id(slug):
    """Get course id from course slug.

    Gets course id from course slug. Throws exception if request fails.
    
    Parameters
    ----------
    slug : str
        Course slug.
        
    Returns
    -------
    course_id : str
        Course id.    
    """
    response = requests.get(course_endpoint_url.format(slug))
    if response.status_code != 200:
        logging.error("Failed to get course id for slug: {}".format(slug))
        raise Exception("Failed to get course id for slug: {}".format(slug))
    course_id = response.json()["elements"][0]["id"]
    return course_id

def safe_get_course_id_request(slug, delay=1, max_retries=5):
    """Get course id from course slug with exception handling.

    Gets course id from course slug and handles exceptions. If 
    exception occurs, it will retry the request after `delay` seconds.
    After `max_retries` retries, it will return None.
    
    Parameters
    ----------
    slug : str
        Course slug.

    delay : int
        Delay in seconds before retrying the request.
        
    Returns
    -------
    course_id : str
        Course id.    
    """
    for _ in range(max_retries):
        try:
            return get_course_id(slug)
        except Exception as e:
            logging.error("Failed to get course id for slug: {}. Retrying after {} seconds".format(slug, delay))
            time.sleep(delay)
    return None


In [ ]:
def tqdm_threadpool_map(func, iterable, no_threads, iterable_length, *args):
    """A threadpool map function that shows a progress bar.

    Parameters
    ----------
    func : function
        The function to apply to each element of the iterable.

    iterable : iterable
        The iterable to apply the function to.

    no_threads : int
        The number of threads to use.

    iterable_length : int
        The length of the iterable.

    *args : list
        The list of arguments to pass to the function.

    Returns
    -------
    results : list
        The list of results from the function.
    """
    with ThreadPoolExecutor(max_workers=no_threads) as executor:
        results = list(tqdm(executor.map(func, iterable, *args), total=iterable_length))
    return results

In [ ]:
course_url = get_all_courses()
print("Total courses: ", len(course_url))

In [ ]:
course_slug = [get_slug(url) for url in course_url]

In [ ]:
course_id = tqdm_threadpool_map(safe_get_course_id_request, course_slug, 4, len(course_slug))

In [ ]:
course_df = pd.DataFrame({"course_url": course_url, "course_slug": course_slug, "course_id": course_id})
course_df.head()

In [ ]:
course_df.to_csv("coursera-course-information.csv", index=False)